### Part 0: Importing Packages

In [ ]:
import requests # getting info from API
import json # type of data we're working with from requests
import pandas as pd # sorting the data values

### Part 1: Working with Requests Library

https://developer.riotgames.com/docs/lol#data-dragon_champions    

Within Riot's API for matches, the champions only have Ids and no names associated with them. \
Before we investigate the matches separately, we need to have a dataFrame of all of the champions associated with their id. \
Here, we will be introducing requests and how to use its functions.

In [ ]:
resp_chmp = requests.get('http://ddragon.leagueoflegends.com/cdn/10.9.1/data/en_US/champion.json')
r3 = resp_chmp.json()
r3 = r3['data']
chmp_name = []
for i in r3:
    chmp_name.append(i)
x = {}
for i in range(len(r3)):
    name = chmp_name[i]
    x[i]= int(r3[name]['key'])
chmp = pd.DataFrame(chmp_name, columns=['Name'])
chmp = chmp.rename(index=x).sort_index()

### Part 2: Extracting Data from API

https://developer.riotgames.com/apis#summoner-v4

**Task**: Retrieve the match numbers, champions, and lane assignments of the account supplied. Then, we'll extract it onto an Excel file

In [ ]:
summonerName = "Phume"
APIKey = "RGAPI-8fb73227-2326-4228-9302-068cc55ffe4d"
URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKey
response = requests.get(URL)
r1 = response.json()
r1['accountId']

In [ ]:
# Returning the accountId of a user using the summonerName and API Key supplied.
# Inputs:
#   summonerName: Name within League of Legends, case and character sensitive
#   APIKey: API Key from Riot Games 
def accId(summonerName, APIKey):
    URL = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + summonerName + "?api_key=" + APIKey
    response = requests.get(URL)
    r1 = response.json()
    return r1['accountId']

In [ ]:
# Find the accountId to get all of the matches from the summoner.
summonerName = (str)(input('Type your Summoner Name here and DO NOT INCLUDE ANY SPACES: '))
APIKey = (str)(input('Copy and paste your API Key here: '))
summ_accId = accId(summonerName, APIKey)
URL = "https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/" + summ_accId + "?api_key=" + APIKey
response = requests.get(URL)
r1 = response.json()
r1 = r1['matches']

In [ ]:
matches = []
champion = []
lane = []
for i in range(len(r1)):
    matches.append(r1[i]['gameId'])
    champ_id = r1[i]['champion']
    champion.append(chmp.loc[champ_id, 'Name'])
    lane.append(r1[i]['lane'])
vals = {'matchId': matches, 'champion': champion, 'lane': lane}
df = pd.DataFrame(vals)

In [ ]:
file_path=r'matches.xlsx'
stats=pd.ExcelWriter(file_path)
df.to_excel(stats)
stats.save()
stats.close()

### Intro to working with Game Data

There are many other pieces of data you can look at. Next week we'll be looking at game data and combining everything we have learned to work with it.

In [ ]:
gameId = "3225011906"
URL_mch = "https://na1.api.riotgames.com/lol/match/v4/matches/" + gameId + "?api_key=" + APIKey
response = requests.get(URL_mch)
r1 = response.json()
r1 = r1['participants']
r1